<a href="https://colab.research.google.com/github/yulbeom/portfolio/blob/main/%EA%B5%90%EC%B0%A8%EA%B2%80%EC%A6%9D%EA%B3%BC_%EA%B7%B8%EB%A6%AC%EB%93%9C_%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
wine = pd.read_csv('http://bit.ly/wine_csv_data')

In [14]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [15]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

In [16]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [17]:
print(sub_input.shape, val_input.shape, test_input.shape)
print(sub_target.shape, val_target.shape, test_target.shape)

(4157, 3) (1040, 3) (1300, 3)
(4157,) (1040,) (1300,)


In [18]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [19]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)  # test_score 5개값을 평균한 것이 교차 검증의 최종 값이다.

{'fit_time': array([0.01002502, 0.00815058, 0.00856805, 0.0085814 , 0.00895834]), 'score_time': array([0.00086474, 0.00076747, 0.0007844 , 0.00076056, 0.00078893]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [20]:
import numpy as np
print(np.mean(scores['test_score'])) # test_score 5개값을 평균한 것이 교차 검증의 최종 값이다.

0.855300214703487


In [21]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold()) # 
print(np.mean(scores['test_score']))

0.855300214703487


In [22]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


# 그리드 서치

In [23]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [24]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001), 
          'max_depth' : range(5, 20, 1),
          'min_samples_split' : range(2, 100, 10)
}
          

In [25]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1) # n_jobs -> 코어 수 , -1 은 끝까지라는 뜻 
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [26]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [27]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [28]:
from scipy.stats import uniform, randint
rgen = randint(0, 10)
rgen.rvs(10)

array([0, 4, 2, 2, 4, 1, 0, 7, 1, 0])

In [29]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 83,  91,  93, 119, 120,  82,  98, 102, 116,  96]))

In [30]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.76065882, 0.50281367, 0.28541904, 0.99658565, 0.01947301,
       0.47026236, 0.22708617, 0.34124712, 0.5308754 , 0.85145645])

In [31]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth' : randint(20,50),
          'min_samples_split' : randint(2, 25),
          'min_samples_leaf' : randint(1, 25),
          }

In [32]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc3fe158890>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc3fe158690>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc3fe2623d0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc3fe158190>},
                   random_state=42)

In [33]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [34]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [35]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
